In [ ]:
MySQL Physical Backup
=============================================

-- this is also called cold backup.
-- this involves physically copying of mysql files into backup device.
-- for this the mysql database server should be clean shutdown.

files required in cold backup
==========================================

-- all the files in datadir.
-- all the files of system tablespaces.
-- all the configuration/option files.

files not required in cold backup
============================================

-- redo log files , undo log files , bin log files 
-- temp files , double write buffer files.


mysql> show variables like '%datadir%';
+---------------+----------------------+
| Variable_name | Value                |
+---------------+----------------------+
| datadir       | /var/lib/mysql/data/ |
+---------------+----------------------+
1 row in set (0.01 sec)

mysql> show variables like '%innodb_data%';
+-----------------------+------------------------------------+
| Variable_name         | Value                              |
+-----------------------+------------------------------------+
| innodb_data_file_path | ibdata1:12M;ibdata2:20M:autoextend |
| innodb_data_home_dir  | /var/lib/mysql/systemdata/         |
+-----------------------+------------------------------------+
2 rows in set (0.01 sec)


clean shutdown the database
=======================================

mysql> set global innodb_fast_shutdown = 0;
Query OK, 0 rows affected (0.00 sec)

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 


copy files to backup location 
=========================================

[root@mysqlserver mysql]# cd /var/lib/mysql
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# ls -rtl
total 8
drwxr-xr-x. 2 mysql mysql   29 May 19 20:38 errorlog
drwxr-xr-x. 2 mysql mysql    6 May 19 20:45 tmpdir
drwxr-xr-x. 2 mysql mysql   81 May 20 00:12 doublewrite
drwxr-xr-x. 3 mysql mysql   26 May 20 02:16 redologs
drwxr-xr-x. 2 mysql mysql   58 May 20 13:22 undolog
drwxr-xr-x. 2 mysql mysql 4096 May 20 15:09 binlog
drwxr-xr-x. 7 mysql mysql 4096 May 20 17:23 data
drwxr-xr-x. 2 mysql mysql    6 May 20 17:23 tempdata
drwxr-xr-x. 2 mysql mysql   58 May 20 17:23 systemdata
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# cp -r /var/lib/mysql/data /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# cp -r /var/lib/mysql/systemdata /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# cp -p /etc/mysql/my.cnf /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# cp -p /var/lib/mysql-files/*.cnf /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# ls -rtl /tmp/mysql_cold_backup/
total 12
-rw-r--r--. 1 root  root   425 May 19 20:47 my.cnf
-rw-r--r--. 1 mysql mysql  691 May 20 15:08 innodb.cnf
drwxr-xr-x. 7 root  root  4096 May 20 17:25 data
drwxr-xr-x. 2 root  root    58 May 20 17:25 systemdata
[root@mysqlserver mysql]# 


cold backup is completed , start the database server 
=========================================================

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 



simulate data corruption by dropping database 
=========================================================

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| posdb              |
| sakila             |
| sys                |
+--------------------+
6 rows in set (0.00 sec)

mysql> drop database sakila;
Query OK, 23 rows affected (0.40 sec)

mysql> drop database posdb;
Query OK, 2 rows affected (0.01 sec)

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+
4 rows in set (0.00 sec)


[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver mysql]#

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]#

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# systemctl start mysqld
Job for mysqld.service failed because the control process exited with error code.
See "systemctl status mysqld.service" and "journalctl -xe" for details.
[root@mysqlserver mysql]# 


[root@mysqlserver mysql]# tail /var/lib/mysql/errorlog/posb_mysqld.log 
2024-05-20T12:07:57.809540Z 0 [ERROR] [MY-010119] [Server] Aborting
2024-05-20T12:07:58.623504Z 1 [ERROR] [MY-011011] [Server] Failed to find valid data directory.
2024-05-20T12:07:58.623952Z 0 [ERROR] [MY-010020] [Server] Data Dictionary initialization failed.
2024-05-20T12:07:58.623981Z 0 [ERROR] [MY-010119] [Server] Aborting
2024-05-20T12:07:58.624477Z 0 [System] [MY-010910] [Server] /usr/sbin/mysqld: Shutdown complete 

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 



Restore from the cold backup taken earlier 
=========================================================

remove all the files before restoring
=========================================

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver mysql]# 

copy the files from backup
=================================

[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/data/* /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# 
[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/systemdata/* /var/lib/mysql/systemdata/
[root@mysqlserver mysql_cold_backup]# 

[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# 
[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/systemdata/
[root@mysqlserver mysql_cold_backup]#

[root@mysqlserver mysql]# systemctl start mysqld
Job for mysqld.service failed because the control process exited with error code.
See "systemctl status mysqld.service" and "journalctl -xe" for details.
[root@mysqlserver mysql]# 

2024-05-20T12:20:18.690519Z 1 [ERROR] [MY-013862] [InnoDB] Neither found #innodb_redo subdirectory, 
nor ib_logfile* files in /var/lib/mysql/redologs/
2024-05-20T12:20:18.690602Z 1 [ERROR] [MY-012930] [InnoDB] Plugin initialization aborted with error Generic error.
2024-05-20T12:20:19.166551Z 1 [ERROR] [MY-010334] [Server] Failed to initialize DD Storage Engine.

[root@mysqlserver mysql]# mkdir /var/lib/mysql/redologs/'#innodb_redo'
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# chown -R mysql:mysql /var/lib/mysql/redologs/
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| posdb              |
| sakila             |
| sys                |
+--------------------+
6 rows in set (0.01 sec)

mysql> use sakila;

mysql> show tables;
+----------------------------+
| Tables_in_sakila           |
+----------------------------+
| actor                      |
| actor_info                 |
| address                    |
| category                   |

mysql> select * from actor;
+----------+-------------+--------------+---------------------+
| actor_id | first_name  | last_name    | last_update         |
+----------+-------------+--------------+---------------------+
|        1 | PENELOPE    | GUINESS      | 2006-02-15 04:34:33 |
|        2 | NICK        | WAHLBERG     | 2006-02-15 04:34:33 |
|        3 | ED          | CHASE        | 2006-02-15 04:34:33 |
|        4 | JENNIFER    | DAVIS        | 2006-02-15 04:34:33 |
|        5 | JOHNNY      | LOLLOBRIGIDA | 2006-02-15 04:34:33 |

In [ ]:
Logical Backups
==============================

-- does not involve copying of physical files.
-- all databases , some databases and table level backup can be taken.
-- utility have to be used like mysqldump amd mysqlpump.
-- takes backup in .sql format.

[mysqluser@mysqlserver backups]$ mysqldump --verbose --help

Dumping structure and contents of MySQL databases and tables.
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases [OPTIONS] DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases [OPTIONS]


Single Table Backup
==================================

[mysqluser@mysqlserver backups]$ mysqldump sakila actor > sakilaDB_actorTable_bkp.sql


Whole Single Database Backup 
===================================

[mysqluser@mysqlserver backups]$ mysqldump sakila > sakilaDB_bkp.sql


Two databases backup
================================

[mysqluser@mysqlserver backups]$ mysqldump --databases posdb sakila > posdb_sakila_bkp.sql
[mysqluser@mysqlserver backups]$


All databases backup 
================================

[mysqluser@mysqlserver backups]$ mysqldump --all-databases > all_dbs_bkp.sql
[mysqluser@mysqlserver backups]$ 


Database backup with skipping some tables
=============================================

[mysqluser@mysqlserver backups]$ mysqldump sakila --ignore-table=sakila.actor --ignore-table=sakila.film_actor > 
sakiladb_skipactor_bkp.sql
[mysqluser@mysqlserver backups]$ 


Multiple database backup with skipping some tables 
=======================================================

[mysqluser@mysqlserver backups]$ mysqldump --databases sakila posdb --ignore-table=sakila.actor 
--ignore-table=sakila.film_actor --ignore-table=posdb.emp_salaries > sakiladb_posdb_ignoretables_bkp.sql
[mysqluser@mysqlserver backups]$ 



Single Table Restore from backup taken for single table
================================================================

[mysqluser@mysqlserver backups]$ cat sakilaDB_actorTable_bkp.sql | grep 'CREATE TABLE'

CREATE TABLE `actor`

[mysqluser@mysqlserver backups]$


mysql> drop table film_actor;
Query OK, 0 rows affected (0.28 sec)

mysql> drop table actor;
Query OK, 0 rows affected (0.01 sec)

mysql> select * from actor;
ERROR 1146 (42S02): Table 'sakila.actor' doesn't exist
mysql> 

[mysqluser@mysqlserver backups]$ mysql sakila < sakilaDB_actorTable_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from actor;
+----------+
| count(*) |
+----------+
|      200 |
+----------+
1 row in set (0.13 sec)



Single Table restore from backup taken for full database
==========================================================

[mysqluser@mysqlserver backups]$ cat sakilaDB_bkp.sql | grep 'CREATE TABLE' | grep film_actor

CREATE TABLE `film_actor` 

[mysqluser@mysqlserver backups]$ 


ALL CREATE TABLE statements start with DROP TABLE IF EXISTS and then CREATE TABALE and then
INSERT STATEMENTS and it will end with keyword UNLOCK TABLES

[mysqluser@mysqlserver backups]$ sed -n -e '/DROP TABLE.*`film_actor`/,/UNLOCK TABLES/p' sakilaDB_bkp.sql 
> sakilaDB_filmactor_bkp.sql
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ mysql sakila < sakilaDB_filmactor_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from film_actor;
+----------+
| count(*) |
+----------+
|     5462 |
+----------+
1 row in set (0.13 sec)


Full database restore from backup taken for a single database
=====================================================================

[mysqluser@mysqlserver backups]$ cat sakilaDB_bkp.sql | grep 'CREATE DATABASE'
[mysqluser@mysqlserver backups]$ 


mysql> drop database sakila;
Query OK, 23 rows affected (0.26 sec)

mysql> select count(*) from sakila.actor;
ERROR 1049 (42000): Unknown database 'sakila'
mysql> 
mysql> create database sakila;
Query OK, 1 row affected (0.19 sec)


[mysqluser@mysqlserver backups]$ mysql sakila < sakilaDB_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from sakila.actor;
+----------+
| count(*) |
+----------+
|      200 |
+----------+
1 row in set (0.13 sec)



2 databases restore from backup taken for 2 databases 
============================================================

[mysqluser@mysqlserver backups]$ cat posdb_sakila_bkp.sql | grep 'CREATE DATABASE'

CREATE DATABASE /*!32312 IF NOT EXISTS*/ `posdb` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci
 */ /*!80016 DEFAULT ENCRYPTION='N' */;
CREATE DATABASE /*!32312 IF NOT EXISTS*/ `sakila` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci 
*/ /*!80016 DEFAULT ENCRYPTION='N' */;

[mysqluser@mysqlserver backups]$ 


mysql> drop database posdb;
Query OK, 2 rows affected (0.08 sec)

mysql> drop database sakila;
Query OK, 23 rows affected (0.23 sec)

[mysqluser@mysqlserver backups]$ mysql < posdb_sakila_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from sakila.actor;
+----------+
| count(*) |
+----------+
|      200 |
+----------+

mysql> select count(*) from posdb.emp_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+


Restore all databases from backup taken for all databases
================================================================

[mysqluser@mysqlserver backups]$ cat all_dbs_bkp.sql | grep 'CREATE DATABASE' | grep posdb
CREATE DATABASE /*!32312 IF NOT EXISTS*/ `posdb` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci 
*/ /*!80016 DEFAULT ENCRYPTION='N' */;
[mysqluser@mysqlserver backups]$


mysql> drop database information_schema;
ERROR 1044 (42000): Access denied for user 'root'@'localhost' to database 'information_schema'
mysql> 
mysql> drop database mysql;
ERROR 3552 (HY000): Access to system schema 'mysql' is rejected.
mysql> 
mysql> drop database performance_schema;
ERROR 1044 (42000): Access denied for user 'root'@'localhost' to database 'performance_schema'
mysql> 
mysql> drop database sys;
Query OK, 101 rows affected (0.20 sec)

mysql> drop database posdb;
Query OK, 2 rows affected (0.01 sec)

mysql> drop database sakila;
Query OK, 23 rows affected (0.23 sec)

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+
3 rows in set (0.00 sec)


[mysqluser@mysqlserver backups]$ mysql < all_dbs_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| posdb              |
| sakila             |
+--------------------+
5 rows in set (0.01 sec)


In [ ]:
Logical Backup - mysqlpump 
=======================================

-- is a newer version and is similar to mysqldump.
-- can use parallelism while backup.
-- compression can be used while backup.
-- user account backups can be taken.
-- shows progress of the backup.


MySQL utility for dumping data from databases to external file.
Usage: mysqlpump [OPTIONS] [--all-databases]
OR     mysqlpump [OPTIONS] --databases DB1 [DB2 DB3...]
OR     mysqlpump [OPTIONS] database [tables]


Use parallelism while taking backup 
============================================

[mysqluser@mysqlserver backups]$ mysqlpump --all-databases --default-parallelism=4 > all_dbs_bkp.sql
Dump progress: 1/1 tables, 0/0 rows
Dump progress: 24/26 tables, 678271/2429177 rows
Dump progress: 24/26 tables, 1455771/2429177 rows
Dump progress: 24/26 tables, 2198271/2429177 rows
Dump completed in 3472
[mysqluser@mysqlserver backups]$ 


[mysqluser@mysqlserver backups]$ mysqlpump --all-databases --default-parallelism=4 --parallel-schemas=3:ordersdb > 
all_dbs_bkp_2.sql
Dump progress: 1/1 tables, 0/0 rows
Dump progress: 24/26 tables, 681271/2429177 rows
Dump progress: 24/26 tables, 1400271/2429177 rows
Dump progress: 24/26 tables, 2092771/2429177 rows
Dump completed in 3865
[mysqluser@mysqlserver backups]$ 


Take user account backup 
============================================

[mysqluser@mysqlserver backups]$ mysqlpump --exclude-databases=% --users > user_account_bkp.sql
Dump completed in 144
[mysqluser@mysqlserver backups]$ 


Take compress backup with lZ4 or ZLIB
===============================================

[mysqluser@mysqlserver backups]$ mysqlpump --all-databases --compress-output=LZ4 > all_dbs_compressbkp.sql.lz4
Dump progress: 1/2 tables, 0/2 rows
Dump progress: 24/26 tables, 676271/2429177 rows
Dump progress: 24/26 tables, 1383021/2429177 rows
Dump progress: 24/26 tables, 2111271/2429177 rows
Dump completed in 3648
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ du -sh all_dbs_bkp.sql
129M	all_dbs_bkp.sql
[mysqluser@mysqlserver backups]$ du -sh all_dbs_compressbkp.sql.lz4
25M	all_dbs_compressbkp.sql.lz4
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ which lz4
/usr/bin/lz4
[mysqluser@mysqlserver backups]$ 
[mysqluser@mysqlserver backups]$ lz4 --help

[mysqluser@mysqlserver backups]$ lz4 -d all_dbs_compressbkp.sql.lz4 all_dbs_compressbkp.sql
all_dbs_compressbkp. : decoded 135102235 bytes                                 
[mysqluser@mysqlserver backups]$ 

In [ ]:
mysqldump or mysqlpump consistent backup 
====================================================

--single-transaction 
                      Creates a consistent snapshot by dumping all tables in a
                      single transaction. Works ONLY for tables stored in
                      storage engines which support multiversioning (currently
                      only InnoDB does); the dump is NOT guaranteed to be
                      consistent for other storage engines.

--skip-lock-tables 


[mysqluser@mysqlserver backups]$ mysqlpump --single-transaction > all_dbs_consistent_bkp.sql
Dump progress: 1/1 tables, 0/0 rows
Dump progress: 24/26 tables, 637521/2284901 rows
Dump progress: 24/26 tables, 1315521/2284901 rows
Dump progress: 24/26 tables, 1974271/2284901 rows
Dump completed in 4040
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ mysqldump --all-databases --single-transaction --skip-lock-tables > 
all_dbs_consistent_bkp_dump.sql
[mysqluser@mysqlserver backups]$ 
